In [ ]:
import numpy as np
import torch
import gpytorch
import matplotlib.pyplot as plt
import matplotlib as mpl 
import plotting_utilities

from rllib.agent.bandit.gp_ucb_agent import GPUCBPolicy
from rllib.environment.bandit_environment import BanditEnvironment
from rllib.reward.gp_reward import GPBanditReward
from rllib.util import rollout_agent
from rllib.util.gaussian_processes import ExactGP
from rllib.util.gaussian_processes.utilities import add_data_to_gp

plotting_utilities.set_figure_params(serif=True)
from matplotlib import rc
rc('font',**{'family':'serif','serif':['Palatino']})
plt.rcParams['pdf.fonttype'] = 42

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
def plot_gp(x: torch.Tensor, model: gpytorch.models.GP, num_samples: int, ax: mpl.axes.Axes) -> None:
    """Plot 1-D GP.

    Parameters
    ----------
    x: points to plot.
    model: GP model.
    num_samples: number of random samples from gp.
    ax: axes where to plot the plot.
    """
    with torch.no_grad(), gpytorch.settings.fast_pred_var():
        pred = model(x)
        mean = pred.mean.numpy()
        error = 2 * pred.stddev.numpy()
        true_values = objective(None, x, None)[0].numpy()

    # Plot gp prediction
    ax.fill_between(x, mean - error, mean + error, lw=0, alpha=0.4, color='C0')
        
    # Plot mean
    ax.plot(x, mean, color='C0')
    
    # Plot ground-truth
    ax.plot(x, true_values, '--', color='k')
    
    # Plot data
    ax.plot(model.train_inputs[0].numpy(),
            model.train_targets.numpy(),
            'x', markeredgewidth=2, markersize=5, color='C1')

    # Plot samples.
    for _ in range(num_samples):
        ax.plot(x.numpy(), pred.sample().numpy())
    
    ax.set_xlim(x[0], x[-1])
    ax.set_ylim(-2.1, 2.1)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_xlabel(r"Inputs $\theta$")
    ax.set_ylabel(r"$J(\theta)$")
        

In [ ]:
# Define objective function
X = torch.tensor([-1., 1., 2.5, 4., 6])
Y = 2 * torch.tensor([-0.5, 0.3, -0.2, .6, -0.5])

NUM_POINTS = 1000
x = torch.linspace(-1, 6, NUM_POINTS)
likelihood = gpytorch.likelihoods.GaussianLikelihood()
likelihood.noise_covar.noise = 0.1 ** 2
objective_function = ExactGP(X, Y, likelihood)
objective_function.eval()
objective = GPBanditReward(objective_function)
environment = BanditEnvironment(objective, x_min=x[[0]].numpy(), x_max=x[[-1]].numpy())

fig, ax = plt.subplots(1, 1)
fig.set_size_inches(5.5/2, 1.5)
with torch.no_grad():
    ax.plot(x, objective_function(x).mean, 'k--')
ax.set_xlabel(r"Inputs $\theta$")
ax.set_ylabel(r"$J(\theta)$")
plt.show()

In [ ]:
def get_new_policy(beta=2.0, noisy=False):
    x0 = x[x > 0.2][[0]].unsqueeze(-1)
    y0 = objective(None, x0, None)[0].type(torch.get_default_dtype())
    gp = ExactGP(x0, y0, likelihood)
    policy = GPUCBPolicy(gp, x, beta=beta, noisy=noisy)
    return policy 

In [ ]:
policy = get_new_policy()
fig, ax = plt.subplots(1, 1)
fig.set_size_inches(5.5/2, 1.5)
plot_gp(x, policy.gp, num_samples=0, ax=ax)

Without optimism (beta=0) and with no noise in the optimization process, GP-UCB gets stuck evaluating the first observation with expected value larger than the mean. By design, this is the case for the first observation.

In [ ]:
policy = get_new_policy(beta=0.0)

for i in range(10):
    query_x = policy(None)[0]
    _, query_y, _, _ = environment.step(query_x)
    add_data_to_gp(policy.gp, query_x.unsqueeze(-1), torch.tensor(query_y, dtype=torch.float))
    
fig, ax = plt.subplots(1, 1)
fig.set_size_inches(5.5 / 2.2, 2)
plot_gp(x, policy.gp, num_samples=0, ax=ax)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
fig.tight_layout(pad=0.2)
plt.savefig('bandit_exploration_beta_0_b.pdf')
plt.show()

If the optimization process is noisy, this method at least converges to a local optimum by exploiting gradient information provided by the slighly randomized evaluations.


In [ ]:
policy = get_new_policy(beta=0.0, noisy=True)

for i in range(10):
    query_x = policy(None)[0]
    _, query_y, _, _ = environment.step(query_x)
    add_data_to_gp(policy.gp, query_x.unsqueeze(-1), torch.tensor(query_y, dtype=torch.float))
    
fig, ax = plt.subplots(1, 1)
fig.set_size_inches(5.5 / 2.2, 2)
plot_gp(x, policy.gp, num_samples=0, ax=ax)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
fig.tight_layout(pad=0.2)
plt.savefig('bandit_exploration_beta_noisy_b.pdf')
plt.show()

With optimism (beta=2), GP-UCB converges to the global optimum as one would expect.

In [ ]:
policy = get_new_policy(beta=2.0)

for i in range(10):
    query_x = policy(None)[0]
    _, query_y, _, _ = environment.step(query_x)
    add_data_to_gp(policy.gp, query_x.unsqueeze(-1), torch.tensor(query_y, dtype=torch.float))

fig, ax = plt.subplots(1, 1)
fig.set_size_inches(5.5 / 2.2, 2)
plot_gp(x, policy.gp, num_samples=0, ax=ax)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
fig.tight_layout(pad=0.2)
plt.savefig('bandit_exploration_beta_2_b.pdf')
plt.show()